In [3]:
import pandas as pd
import numpy as np

---

# Loading Data

---

In [10]:
data = pd.read_csv('data/sales_train.csv')
data.shape

(2935849, 6)

#####  Dataset has 2.9 millions sales entries and 6 columns

In [11]:
data.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [13]:
items = pd.read_csv('data/items.csv')
items.shape

(22170, 3)

##### There are 22k unique items referenced in the data

In [14]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [15]:
item_categories = pd.read_csv('data/item_categories.csv')
item_categories.shape

(84, 2)

##### Items are split into 84 categories

In [16]:
item_categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [17]:
shops = pd.read_csv('data/shops.csv')
shops.shape

(60, 2)

In [18]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


##### Sales are distributed across 60 shops

---

# Exploration

---